In [ ]:
import os 

# List all subdirectories in data/borealis/domains
target_folder = './data/borealis/domains'
data_directories = [d for d in os.listdir(target_folder) if os.path.isdir(os.path.join(target_folder, d))]
print(data_directories)

In [9]:
# Solve the problems with Fast Downward to generate the ground truth plans
import os
import subprocess
from src.llm_plan import config as cfg

# Config (from your config module)
solver_fd_binary = cfg.SOLVER_FD_BINARY           # e.g. "./solvers/fast-downward-24.06.1/fast-downward.py"

num_problems = 20
problem_name = "p{num}.pddl"  # zero-padded to match your original

for data_directory in data_directories:
    print(f"Processing data directory: {data_directory}")
    data_path = os.path.join(target_folder, data_directory)
    domain_path = os.path.join(data_path, "domain.pddl")

    for i in range(1, num_problems + 1):
        num_str = f"0{i}" if i < 10 else str(i)
        problem_path = os.path.join(data_path, problem_name.format(num=num_str))
        sas_filename = "sas_file" + num_str
        sas_plan_path = os.path.join(data_path, sas_filename)
        print(f"\tProcessing problem {i} in {data_path}")
        cmd = (
            f'{solver_fd_binary} '
            f'--alias lama-first '
            f'--search-time-limit 300 '
            f'--plan-file {sas_plan_path} '
            f'{domain_path} '
            f'{problem_path} '
            # f'--search "astar(lmcut())"'. # for optimization but it takes too long
        )
        proc = subprocess.run(cmd, shell=True, capture_output=True, text=True)
        print(f"\tSaving results to {sas_plan_path}.")

Processing data directory: floortile
	Processing problem 1 in ./data/borealis/domains/floortile
	Saving results to ./data/borealis/domains/floortile/sas_file01.
	Processing problem 2 in ./data/borealis/domains/floortile
	Saving results to ./data/borealis/domains/floortile/sas_file02.
	Processing problem 3 in ./data/borealis/domains/floortile
	Saving results to ./data/borealis/domains/floortile/sas_file03.
	Processing problem 4 in ./data/borealis/domains/floortile
	Saving results to ./data/borealis/domains/floortile/sas_file04.
	Processing problem 5 in ./data/borealis/domains/floortile
	Saving results to ./data/borealis/domains/floortile/sas_file05.
	Processing problem 6 in ./data/borealis/domains/floortile
	Saving results to ./data/borealis/domains/floortile/sas_file06.
	Processing problem 7 in ./data/borealis/domains/floortile
	Saving results to ./data/borealis/domains/floortile/sas_file07.
	Processing problem 8 in ./data/borealis/domains/floortile
	Saving results to ./data/borealis/d

In [18]:
# Generate the dataset for our experiments
import json
import os

num_problems = 20
problem_pddl_name = "p{num}.pddl"  # zero-padded to match your original
problem_nl_name = "p{num}.nl"

domain_template_name = "domain_template.pddl"
domain_nl_name = "domain.nl"

prompt = """You are an expert planner. I will provide you with:
- A natural language description of the domain.
- A natural language description of the problem. 
- A PDDL domain template.
- A PDDL problem template.

Here are:
- The natural language description of the domain:
<domain_nl>
{domain_nl}
</domain_nl>

- The natural language description of the problem:
<problem_nl>
{problem_nl}
</problem_nl>

- The PDDL problem template:
<problem_pddl>
{problem_pddl}
</problem_pddl>

- The PDDL domain template:
<domain_template>
{domain_template}
</domain_template>

Your task is to generate a a PDDL domain and a PDDL problem that accurately represents the scenario described above, so that it is possible to solve the task with the FastDownwards solver.
"""

for data_directory in data_directories:
    print(f"Processing data directory: {data_directory}")
    data_path = os.path.join(target_folder, data_directory)
    domain_template_path = os.path.join(data_path, domain_template_name)
    domain_nl_path = os.path.join(data_path, domain_nl_name)
    
    # Instantiate the results dictionary
    results = {}

    for i in range(1, num_problems):
        num_str = f"0{i}" if i < 10 else str(i)
        problem_nl_path = os.path.join(data_path, problem_nl_name.format(num=num_str))
        problem_pddl_path = os.path.join(data_path, problem_pddl_name.format(num=num_str))

        if not os.path.exists(os.path.join(data_path, f"sas_file{num_str}")):
            sas_plan_path = "UNSOLVABLE"
        else:
            sas_plan_path = open(os.path.join(data_path, f"sas_file{num_str}")).read()

        results[f"{data_directory}_example_{i-1}"] = {
            "prompt_0shot": prompt.format(
                domain_nl=open(domain_nl_path).read(),
                problem_nl=open(problem_nl_path).read(),
                problem_pddl=open(problem_pddl_path).read(),
                domain_template=open(domain_template_path).read()
            ),
            "golden_plan": sas_plan_path
        }
    
    # Save the results to a file
    output_path = os.path.join(data_path, f"{data_directory}.json")
    with open(output_path, 'w') as f:
        json.dump(results, f, indent=4)
        

Processing data directory: floortile
Processing data directory: childsnack
